In [114]:
#Read Input & Assign Variables
import csv
with open('toy_testcase.csv', newline='') as input:
    data = tuple(csv.reader(input))

M = int(data[0][0])
N = int(data[0][1])
K = int(data[1][0])
alpha = int(data[2][0])

init_speed = ()
data_size = ()
factor = ()

for i in range (3,3+K):
    init_speed = (*init_speed, int(data[i][1]))
    data_size = (*data_size, int(data[i][2]))
    factor = (*factor, int(data[i][3]))
print(factor)
total_data = sum(list(data_size))
data_remaining = list(data_size)

(30, 25, 60, 40)


In [115]:
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed = []
for i in range (0,len(speed_map)):
    speed = (*speed, int(speed_map[i][1]))


In [116]:
def size_index(data):
    index_map = []
    for i in range (0,K):
        max_index = data.index(max(data))
        index_map.append(max_index)
        data[max_index] = -1
    return index_map

In [117]:
def target_factor(grid_map,users):
    current_factor = 0
    collocated_factor_list = [0]*M
    for i in range (0,users):
        collocated_factor_list[i] = factor[grid_map[i]-1]/100
    
    for i in range (0,users):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))

    return current_factor

In [118]:
target_ideal = 100/N
grid_map_vertical = [0]*M

for i in range (0,N):       #Columns
    size_index_map = size_index(data_remaining)         #shows the index of largest and the subsequent largest data_size
    for j in range (0,M):
        grid_map_vertical[j] = size_index_map[j]+1
        target_curr = target_factor(grid_map_vertical,j+1) 
        

0.0
0.24
0.24
0.34
0.39
0.25
0.0
0.09
0.09
0.18
0.18
0.18
0.0
0.09
0.09
0.18
0.18
0.18
0.0
0.09
0.09
0.18
0.18
0.18
0.0
0.09
0.09
0.18
0.18
0.18
